In [1]:
# identifier: yt-cleaned-data
# Master username: admin
# Master password: Admin123
# new vpc security group name: yt_demo_vpc
# Database port: 3306
# initial database name: yt_cleaned_data

In [2]:
pip install pandas sqlalchemy mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/5f/c3/b4c36fbc65c119fca83f258557573d8cd296da5e198d3752c669f8e871e2/mysql_connector_python-9.0.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB 165.2 kB/s eta 0:01:27
   ---------------------------------------- 0.0/14.3 MB 219.4 kB/s eta 0:01:06
   ---------------------------------------- 0.1/14.3 MB 327.3 kB/s eta 0:00:44
   ---------------------------------------- 0.1/14.3 MB 502.3 kB/s eta 0:00:29
    --------------------------------------- 0.3/14.3 MB 853.3 kB/s eta 0:00:17
   - -------------------------------------- 0.5/14.3 MB 1.2 MB/s eta 0:00:12
   - -------------------------------------- 0.5/14.3 MB 1.2 MB/s eta 0:00:12
   

In [3]:
pip install nltk

In [4]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary datasets
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Get English stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Lemmatize and remove stopwords
    processed_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]

    # Join the processed words back into a string
    return ' '.join(processed_words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import json
import boto3
from botocore.exceptions import NoCredentialsError

# AWS S3 credentials
aws_access_key_id = ''  # Replace with your actual AWS Access Key ID
aws_secret_access_key = ''  # Replace with your actual AWS Secret Access Key

# Initialize a session using Amazon S3
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,  # Remove or comment out if not needed
    aws_secret_access_key=aws_secret_access_key  # Remove or comment out if not needed
)

# Bucket name and file name
bucket_name = 'youtuberawdata1'
file_name = 'youtube_raw_data.json'  # Replace with the name of your file in the bucket

# Download the file from S3
try:
    response = s3.get_object(Bucket=bucket_name, Key=file_name)
    content = response['Body'].read().decode('utf-8')
    # Convert JSON content to Python object (list of dictionaries)
    json_content = json.loads(content)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(json_content)
    
except Exception as e:
    print(f"Error: {e}")

In [7]:
import pandas as pd
# df = pd.read_excel('youtube_data.xlsx')
# df = pd.read_excel("/content/drive/MyDrive/Guvi Final Project/youtube_data.xlsx")

tages_list = list(df['tags_list'])
title_list = list(df['title_list'])
video_url_list = list(df['video_url_list'])
# video_url_list_cleaned = []
# for i in video_url_list:
#   video_url_list_cleaned.append('https://www.youtube.com/watch?v=' + i.split('?')[1])
# df['video_url_list'] = video_url_list_cleaned

def tags_cleaning(x):
  x = x.replace('[', '')
  x = x.replace(']', '')
  x = x.replace("'", '')
  x = x.split(', ')
  return x

tags_list_cleaned = []
for i in tages_list:
  tags_list_cleaned.append(tags_cleaning(i))

# Combine the tags into a single string for each list
tags_list_cleaned = [" ".join(tags) for tags in tags_list_cleaned]


df['tags_list'] = tags_list_cleaned

tags_list = list(df["tags_list"])

# df.to_excel('youtub_data_labelled_cleaned.xlsx', index = False)
# df.to_json('youtub_data_labelled_cleaned.json', orient='records', lines=True)
video_url_list = list(df['video_url_list'])

In [8]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
host = ''  # Replace with your actual AWS RDS endpoint
database = 'yt_cleaned_data'
user = 'admin'
password = 'Admin123'
port = 3306

# Create SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

# Create table from DataFrame
df.to_sql(name='yt_cleaned_data', con=engine, if_exists='replace', index=False)

print("Table 'videos' created successfully from DataFrame")

Table 'videos' created successfully from DataFrame


In [9]:
# Read data from MySQL table into a DataFrame
df = pd.read_sql('SELECT * FROM yt_cleaned_data', con=engine)

In [10]:
df

,video_url_list,publishedAt_list,title_list,tags_list,viewCount_list,likeCount_list
0,https://www.youtube.com/watch?v=Wz2r91H_X_8,2017-06-15T14:57:49Z,My Practical Experience from Macbook Air to Ma...,macbook pro 13 vs macbook air macbook pro 13 t...,180011,3053
1,https://www.youtube.com/watch?v=Ty6ajzVMvVY,2022-07-10T10:41:00Z,I tested Crazy Fitness Gadgets vs Ryan Trahan,fitness gadgets gym workout Ryan trahan cool t...,4685053,163250
2,https://www.youtube.com/watch?v=zfy222DUsK0,2017-11-21T13:34:48Z,Oneplus 5T vs Pixel 2 XL vs S8 vs iPhone 8 Bat...,oneplus 5t charging test oneplus 5t charge tes...,418514,7632
3,https://www.youtube.com/watch?v=dCNlL5i1dgQ,2019-11-07T15:43:35Z,Top 10 Certifications For 2020 | High Paying I...,yt:cc=on edureka Top 10 Certifications For 202...,500421,13454
4,https://www.youtube.com/watch?v=bYMUb4uQZoo,2015-11-01T11:17:54Z,DO WHAT IS HARD - Motivational Video,Motivational video motivational speech motivat...,2847948,26972
...,...,...,...,...,...,...
4202,https://www.youtube.com/watch?v=aDgwhMjsiSM,2024-08-03T16:23:30Z,Pune Court Grants Bail To Puja Khedkar's Mothe...,manorama khedkar ias pooja manorama khedkar po...,553,2
4203,https://www.youtube.com/watch?v=_YtquNBscvk,2024-06-18T02:16:03Z,Kanchanjunga Express | Human Error In Bengal T...,NDTV 24x7 breaking news current affairs latest...,8285,29
4204,https://www.youtube.com/watch?v=b61TGA1s2Ok,2024-06-13T10:04:59Z,"Iran and UAE drones used in Sudan war, evidenc...",bbc bbc news news world news breaking news us ...,33805,552
4205,https://www.youtube.com/watch?v=zwOe69BufqU,2024-01-09T08:18:44Z,Best Flagship SoC of 2024 Battle - Snapdragon ...,Trakintech trakin Tech English Snapdragon 8 Ge...,105680,3445
